# CNN using Pytorch

This is a basic CNN template code for anyone to use!

In [ ]:
# @title Basic convolutional neural network code in Pytorch #opensource
# @markdown Author: Marc Haraoui

# @markdown Year of update: 2023

### Imports and versions

In [1]:
! python --version

Python 3.10.12


In [1]:
!nvidia-smi

Wed Nov 22 12:32:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch

# Check if cuda is available
print(f"Is cuda available? {torch.cuda.is_available()}")

Is cuda available? True


## DIY CNN with Pytorch

### CNN Class

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class BasicCNN(nn.Module):
  def __init__(self):
    super(BasicCNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # 1 channel (input) to 10 channels (output)
    self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # 10 to 20
    self.fc1 = nn.Linear(320, 50) # Fully connected MLP with 320 inputs and 50 outputs
    self.fc2 = nn.Linear(50, 10) #  MLP: 50 inputs et 10 output classes

  def forward(self, x):
    out = self.conv1(x)
    out = F.max_pool2d(out, 2) # downsampling operation
    out = F.relu(out) # relu activation function
    out = F.relu(F.max_pool2d(self.conv2(out), 2)) # same as three previous lines
    out = out.view(-1, 320)
    out = F.relu(self.fc1(out))
    out = self.fc2(out)
    return F.log_softmax(out, dim=1) # last activation function (probability distribution)

### MNIST Dataset

In [4]:
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

# Numpy array to pytorch tensor
transform = transforms.ToTensor()

# Get MNIST dataset for training
trainset = torchvision.datasets.MNIST(root='./data', train=True,
download=True, transform=transform)

# Create the train loader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
shuffle=True, num_workers=2)

# Get MNIST dataset for test
testset = torchvision.datasets.MNIST(root='./data', train=False,
download=True, transform=transform)

#Create the test loader
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
shuffle=False, num_workers=2)

100%|██████████| 9912422/9912422 [00:00<00:00, 101842094.77it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 72666882.92it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 27422141.06it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 23519171.32it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



### Training our CNN

#### Model init and SGD optimizer

-> You can use another one such as Adam or variations.

In [6]:
cuda = torch.device('cuda')

# Initializing the model on cuda
our_model = BasicCNN().to(cuda)

# SGD optimizer with a e-2 learning rate (you can use a scheduler if necessary)
optimizer = optim.SGD(our_model.parameters(), lr=0.01, momentum=0.9)

#### Train and Test functions

In [10]:
def train(model, loader, optimizer, epoch, cuda=cuda):
  model.train()
  for batch_idx, (data, target) in enumerate(loader):
    data = data.to(cuda)
    target = target.to(cuda)
    optimizer.zero_grad()

    # Predictions per batch
    output = model(data)

    # Loss
    loss = F.cross_entropy(output, target)
    loss.backward() # back propagation
    optimizer.step()

    # Print result and save checkpoint every 10 batch
    if batch_idx % 10 == 0:
      print('Train epoch {}: [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data),
            len(loader.dataset),100. * batch_idx / len(loader), loss.item()))

def test(model, loader, cuda=cuda):
  model.eval()
  loss = 0
  correct = 0
  with torch.no_grad(): # no gradient in eval mode
    for data, target in loader:
      data = data.to(cuda)
      target = target.to(cuda)

      # Predictions per batch
      output = model(data)

      # Loss
      loss += F.nll_loss(output, target, reduction='sum').item() # negative log likelihood loss
      prediction = output.max(1, keepdim=True)[1]
      correct += prediction.eq(target.view_as(prediction)).sum().item()
      loss /= len(loader.dataset)

      # Print results
      print('Test: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            loss, correct, len(loader.dataset), 100. * correct / len(loader.dataset)))

#### Train and test the model for 3 epochs

In [11]:
nb_epochs = 3

for epoch in range(nb_epochs):
    train(our_model, trainloader, optimizer, epoch, cuda)
    test(our_model, testloader, cuda)

Train epoch 0: [0/60000 (0%)]	Loss: 0.019460
Train epoch 0: [320/60000 (1%)]	Loss: 0.009177
Train epoch 0: [640/60000 (1%)]	Loss: 0.019114
Train epoch 0: [960/60000 (2%)]	Loss: 0.000214
Train epoch 0: [1280/60000 (2%)]	Loss: 0.000275
Train epoch 0: [1600/60000 (3%)]	Loss: 0.007748
Train epoch 0: [1920/60000 (3%)]	Loss: 0.004060
Train epoch 0: [2240/60000 (4%)]	Loss: 0.006806
Train epoch 0: [2560/60000 (4%)]	Loss: 0.012380
Train epoch 0: [2880/60000 (5%)]	Loss: 0.013040
Train epoch 0: [3200/60000 (5%)]	Loss: 0.002067
Train epoch 0: [3520/60000 (6%)]	Loss: 0.059228
Train epoch 0: [3840/60000 (6%)]	Loss: 0.023873
Train epoch 0: [4160/60000 (7%)]	Loss: 0.000214
Train epoch 0: [4480/60000 (7%)]	Loss: 0.003368
Train epoch 0: [4800/60000 (8%)]	Loss: 0.062612
Train epoch 0: [5120/60000 (9%)]	Loss: 0.051413
Train epoch 0: [5440/60000 (9%)]	Loss: 0.155081
Train epoch 0: [5760/60000 (10%)]	Loss: 0.033838
Train epoch 0: [6080/60000 (10%)]	Loss: 0.003122
Train epoch 0: [6400/60000 (11%)]	Loss: 0.00

#### Save checkpoint

In [ ]:
# If you want to save ckpt, please add a path first
torch.save({
    'epoch': nb_epochs,
    'model_state_dict': our_model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    }, "INSERT_PATH")

### Retrain/Fine-tune our CNN

#### Load checkpoint

In [ ]:
# To load the checkpoint, use the 'torch.load()' function. Here's an example:
new_model = BasicCNN() # load a new model with the same structure
new_optimizer = optim.SGD(new_model.parameters(), lr=0.01, momentum=0.9)

#retrieve and load checkpoint onto the new model and optimizer
checkpoint = torch.load("INSERT_PATH")
new_model.load_state_dict(checkpoint['model_state_dict'])
new_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
continue_epoch = checkpoint['epoch']

#### Continue training model or fine-tune

In [ ]:
nb_epochs = 1

for epoch in range(nb_epochs):
    train(new_model, trainloader, new_optimizer, epoch, cuda)
    test(new_model, testloader, cuda)